In [57]:
# Import dependencies
import numpy as np
import matplotlib.pyplot as plt
from numpy import loadtxt
import pandas as pd
from tqdm import tqdm
from pprint import pprint
import requests
import json
import csv
import math
from datetime import datetime
from dateutil.relativedelta import relativedelta
import statistics

# Importing sector, price, financials, & market data

In [58]:
# Get the organCode from list of all tickers
def process(filename:str="/Users/nambuismbp/projects/investment_research/data/codedict.txt") -> dict:
    ticker_to_organ_code = {}
    with open(filename, "r") as json_file:
        data = json.load(json_file)
        ticker_to_organ_code = {
            item["ticker"]: item["organCode"] for item in data["items"]}
        return ticker_to_organ_code
    return None

ticker_dict = process()

# Get ticker from the organCode
def get_ticker(val):
    for key, value in ticker_dict.items():
        if val == value:
            return key

In [59]:
# Load in sector data
dfSectorInfo = pd.read_csv(r'/Users/nambuismbp/projects/investment_research/data/descriptive_info.csv')

In [60]:
# Load in pricing data
dfPricingInfo = pd.read_csv(r'/Users/nambuismbp/projects/investment_research/data/pricing_data.csv')

# Uniform column name to be ticker
for col in dfPricingInfo:
    dfPricingInfo.rename(columns = {col: get_ticker(col)}, inplace = True)

# Create new monthly date series and insert into dataframe
monthSeries = pd.date_range('2008-01-31', periods=len(dfPricingInfo), freq='M')
count = 0
for date in reversed(range(len(monthSeries))):
    dfPricingInfo.at[count, 'date'] = monthSeries[date]
    count = count + 1

# Rearrange columns so that date appears first
del dfPricingInfo[dfPricingInfo.columns[0]]
cols = dfPricingInfo.columns.tolist()
cols = cols[-1:] + cols[:-1]
dfPricingInfo = dfPricingInfo[cols]

In [61]:
# Load in trading value data
dfTradingVal = pd.read_csv(r'/Users/nambuismbp/projects/investment_research/data/trading_value_data.csv')

# Put trading value data having the same dates as pricing data
del dfTradingVal[dfTradingVal.columns[0]]
extracted = dfPricingInfo['date'].to_list()
dfTradingVal.insert(0, "date", extracted)

# Uniform column name to be ticker
for col in dfTradingVal:
    if col != 'date': dfTradingVal.rename(columns = {col: get_ticker(col)}, inplace = True)

In [62]:
# Load in financial data
with open("/Users/nambuismbp/projects/investment_research/data/financial_data_dec29.txt", 'r') as file:
    financials_raw = json.load(file)

financials={}
for (key, value) in financials_raw.items():
    ticker = get_ticker(key)
    financials[ticker] = value

In [63]:
# Load in market return
dfMarketReturn = pd.read_csv(r'/Users/nambuismbp/projects/investment_research/data/market_return.csv')
monthSeries = pd.date_range('2000-07-31', periods=len(dfMarketReturn), freq='M')
count = 0
for date in reversed(range(len(monthSeries))):
    dfMarketReturn.at[count, 'date'] = monthSeries[date]
    count = count + 1

# Scale up returns to match display format of stock & portfolios
dfMarketReturn['market_return'] = dfMarketReturn['market_return'].multiply(100)

# Functions to select the right universe based on data availability & liquidity


In [64]:
# Select only companies that has all 4 quarters of financial data in the previous year
def check_financial_data(date, data) -> list:
    financialScreeningList = []
    for ticker in data:
        count = data[ticker]['years'].count(date.year - 1)
        if count == 4: financialScreeningList.append(ticker)
    return financialScreeningList

In [65]:
# Select only companies that have pricing data in the month
def check_pricing_data(date, data) -> list:
    priceScreeningList = []
    index = data[data['date'] == datetime(date.year, date.month, date.day)].index[0]
    for ticker in dfPricingInfo:
        if ticker != 'date' and not math.isnan(dfPricingInfo.loc[index, ticker]):
            priceScreeningList.append(ticker)
    return priceScreeningList

In [66]:
# Select only companies that are not banks, insurance or financial services
def check_sector_data(dataframe) -> list:
    dataRemove = dataframe[dataframe['ICB_Sector_L2'] != 'Dịch vụ tài chính L2']
    dataRemove = dataRemove[dataRemove['ICB_Sector_L2'] != 'Ngân hàng L2']
    dataRemove = dataRemove[dataRemove['ICB_Sector_L2'] != 'Bảo hiểm L2']
    
    sectorScreeningList = dataRemove['Ticker'].to_list()
    return sectorScreeningList

# Functions to rank and sort stocks by factors


In [67]:
# Value factor = Book Value of Last Accounting Year / Current Market Cap
def get_value(date, tickers, financials) -> dict:
    quarter = (date.month-1)//3 + 1
    valueFactorDict = {}
    for ticker in tickers:
        # Getting current Market Cap
        currDateIndex = 0
        currNumShare = 0
        numShare = 0
        marketCap = 0
        for i in range(len(financials[ticker]['years'])):
            if financials[ticker]['years'][i] == date.year and financials[ticker]['quarters'][i] == quarter:
                currDateIndex = i
        if currDateIndex != 0:
            numShare = financials[ticker]['num_share'][currDateIndex]
        
        currPriceIndex = dfPricingInfo[dfPricingInfo['date'] == date].index[0]
        currPrice = dfPricingInfo.loc[currPriceIndex, ticker]
        
        if numShare != 0 and currPrice != 0 and not np.isnan(currPrice) and not np.isnan(numShare):
            marketCap = numShare * currPrice
        
        # Getting last year book value
        bookValue = 0
        lastYearIndex = 0
        for i in range(len(financials[ticker]['years'])):
            if financials[ticker]['years'][i] == date.year-1 and financials[ticker]['quarters'][i] == 4:
                lastYearIndex = i
        if lastYearIndex != 0:
            result = financials[ticker]['total_equity'][lastYearIndex]
            if result is not None:
                bookValue = result
        
        # Calculate BE / ME only if the firm does not have negative book value
        if marketCap != 0 and bookValue > 0:
            valueFactorDict[ticker] = math.log(bookValue / marketCap)
    return valueFactorDict     

In [68]:
get_value(datetime(2021, 6, 30), ['AAA', 'AAM', 'AAT', 'ABR', 'ABS'], financials)

{'AAA': -0.2394021587289241,
 'AAM': 0.3847734346087532,
 'AAT': -0.2941687576093243,
 'ABR': -0.6854337888953977,
 'ABS': -1.6328420628492712}

In [69]:
def sort_dict(myDict) -> dict:
    sortedDict = {k: v for k, v in sorted(myDict.items(), key=lambda item: item[1])}
    return sortedDict

# Functions to create decile portfolios and calculate returns

In [70]:
def create_decile_portfolios(sortedDict, decile) -> list:
    keys = pd.Series(list(sortedDict.keys()))
    values = pd.Series(list(sortedDict.values()))
    df = pd.DataFrame({'ticker': keys, 'values': values})
    df['decile_rank'] = pd.qcut(df['values'], numPortfolio, labels = False)
    
    idx = df.index[df['decile_rank'] == decile].tolist()
    decileList = df['ticker'][idx].tolist()
    return decileList

In [71]:
def diff_month(d1, d2) -> int:
    return (d2.year - d1.year) * 12 + d2.month - d1.month

#Function to calculate average monthly return of a stock in month beginmonth + 1
def get_stock_monthly_return(stock, beginmonth) -> float:
    index = dfPricingInfo.index[dfPricingInfo['date'] == beginmonth]
    
    beginPrice = dfPricingInfo.iloc[index][stock].iat[0]
    endPrice = dfPricingInfo.iloc[index-1][stock].iat[0]
    
    monthlyReturn = (endPrice / beginPrice - 1) * 100
    return monthlyReturn

In [72]:
def get_portfolio_monthly_return(portfolio, beginPeriod, endPeriod) -> float:
    monthSeries = pd.date_range(start=beginPeriod, end=endPeriod, freq = 'M')
    portfolioMonthlyReturn = []
    
    for month in monthSeries:
        if month == endPeriod:
            break
        temp = []
        for stock in portfolio:
            temp.append(get_stock_monthly_return(stock, month))
        average = sum(temp) / len(temp)
        portfolioMonthlyReturn.append(average)
        
    return portfolioMonthlyReturn

In [73]:
def get_stock_monthly_trading_val(stock, beginmonth) -> float:
    index = dfTradingVal.index[dfTradingVal['date'] == beginmonth]
    tradingVal = dfTradingVal.iloc[index-1][stock].iat[0]
    return tradingVal

In [74]:
def get_portfolio_yearly_avg_trading_val(portfolio, beginPeriod, endPeriod) -> float:
    monthSeries = pd.date_range(start=beginPeriod, end=endPeriod, freq = 'M')
    portfolioTradingVal = []
    
    for month in monthSeries:
        if month == endPeriod:
            break
        temp = []
        for stock in portfolio:
            temp.append(get_stock_monthly_trading_val(stock, month))
        average = sum(temp) / len(temp)
        portfolioTradingVal.append(average)
    return sum(portfolioTradingVal) / len(portfolioTradingVal)

# Functions to get beta & standard deviation

In [75]:
def get_portfolio_beta(portfolioReturns, market_data) -> list:
    betaList = []
    monthlyRet = []
    marketRet = []
    
    #Calculate beta
    for portfolio in range(numPortfolio):
        tempList = []
        for year in portfolioReturns:
            tempList.extend(portfolioReturns[year][portfolio])
        monthlyRet.append(tempList)

    t = beginDate
    for period in range(len(yearSeries)):
        index = dfMarketReturn[dfMarketReturn['date'] == t].index[0]
        index = index - 1
        for i in range(12):
            marketRet.append(dfMarketReturn.loc[index, 'market_return'])
            index = index - 1 
        t = t.replace(year = t.year + 1)
    
    for i in range(len(monthlyRet)):
        dfMonthlyRet = pd.Series(monthlyRet[i])
        dfMarketRet = pd.Series(marketRet)
        covar = dfMonthlyRet.cov(dfMarketRet)
        var = dfMarketRet.var()
        beta = round(covar / var, 3)
        betaList.append(beta)
    return betaList

In [76]:
def get_portfolio_std(portfolioReturns) -> list:
    monthlyRet = []
    stdList = []
    for portfolio in range(numPortfolio):
        tempList = []
        for year in portfolioReturns:
            tempList.extend(portfolioReturns[year][portfolio])
        monthlyRet.append(tempList)
    for i in range(len(monthlyRet)):
        dfMonthlyRet = pd.Series(monthlyRet[i])
        stdList.append(round(dfMonthlyRet.std(), 3))
    return stdList

# Set parameters for the model

In [77]:
# Set begin and end dates for backtesting
beginDate = datetime(2012, 6, 30)
endDate = datetime(2022, 6, 30)

# Set number of portfolios
numPortfolio = 10

# Set liquidity restriction by value
minLiquidity = 50000000

# Running model

In [78]:
# Generate date series for the backtesting
yearSeries = pd.date_range(start=beginDate, end=endDate, freq = 'Y')

In [79]:
# Get the universe of acceptable stocks for each year
date = beginDate
universe = {}
for year in yearSeries:
    accountingYear = date - relativedelta(years=1)
    financialScreeningList = check_financial_data(accountingYear, financials)
    priceScreeningList = check_pricing_data(date, dfPricingInfo)
    sectorScreeningList = check_sector_data(dfSectorInfo)
    universe[year] = list(set(sectorScreeningList) & set(priceScreeningList) & set(financialScreeningList))    
    
    print(f'In yearly period starting {date} the investable unvierse has {len(universe[year])} stocks')
    date = date.replace(year = date.year + 1)

In yearly period starting 2012-06-30 00:00:00 the investable unvierse has 190 stocks
In yearly period starting 2013-06-30 00:00:00 the investable unvierse has 219 stocks
In yearly period starting 2014-06-30 00:00:00 the investable unvierse has 232 stocks
In yearly period starting 2015-06-30 00:00:00 the investable unvierse has 245 stocks
In yearly period starting 2016-06-30 00:00:00 the investable unvierse has 251 stocks
In yearly period starting 2017-06-30 00:00:00 the investable unvierse has 272 stocks
In yearly period starting 2018-06-30 00:00:00 the investable unvierse has 289 stocks
In yearly period starting 2019-06-30 00:00:00 the investable unvierse has 307 stocks
In yearly period starting 2020-06-30 00:00:00 the investable unvierse has 329 stocks
In yearly period starting 2021-06-30 00:00:00 the investable unvierse has 344 stocks


In [80]:
# Get the decile portfolios based on ranking factor for each year
portfolios = {}

for year in yearSeries:
    keys = universe[year]
    # Get a dictionary of the value of factor for each stock
    factor = get_value(datetime(year.year, beginDate.month, beginDate.day), keys, financials)
    
    # Remove stocks from the ranking list if they are not in the universe
    factor = {key:val for key, val in factor.items() if key in universe[year]}
    
    # Remove stocks that do not meet liquidity restriction of more than 10 million VND per day
    liquidityScreening = []
    for stock in keys:
        yearlyLiquidity = []
        for month in range(12):
            yearlyLiquidity.append(get_stock_monthly_trading_val(stock, datetime(year.year, beginDate.month, beginDate.day)))
        avgMonthlyLiquidity = (sum(yearlyLiquidity) / len(yearlyLiquidity)) / 12
        avgDailyLiquidity = avgMonthlyLiquidity / 21
        if avgDailyLiquidity > minLiquidity:
            liquidityScreening.append(stock)
            
    # Remove stocks from the ranking list if they do not meet liquidity restrictions
    factor = {key:val for key, val in factor.items() if key in liquidityScreening}
    
    # Sort based on factor
    factorSorted = sort_dict(factor)
    
    # List of decile portfolios in a given year
    portfolioList = []
    for decile in range(numPortfolio):
        getPortfolio = create_decile_portfolios(factorSorted, decile)
        portfolioList.append(getPortfolio)
        
        avgScore = []
        
        # Print out average BE/ME for each decile in the year
        for item in getPortfolio:
            avgScore.append(factor[item])
        print(f"The average BE/ME in year {year.year} for portfolio decile {decile + 1} is {round(sum(avgScore)/len(avgScore), 3)}")
        print(f"Number of companies in year {year.year} for portfolio decile {decile + 1} is {len(getPortfolio)}")        

    portfolios[year] = portfolioList

The average BE/ME in year 2012 for portfolio decile 1 is -0.321
Number of companies in year 2012 for portfolio decile 1 is 8
The average BE/ME in year 2012 for portfolio decile 2 is 0.351
Number of companies in year 2012 for portfolio decile 2 is 8
The average BE/ME in year 2012 for portfolio decile 3 is 0.626
Number of companies in year 2012 for portfolio decile 3 is 8
The average BE/ME in year 2012 for portfolio decile 4 is 0.732
Number of companies in year 2012 for portfolio decile 4 is 8
The average BE/ME in year 2012 for portfolio decile 5 is 0.9
Number of companies in year 2012 for portfolio decile 5 is 8
The average BE/ME in year 2012 for portfolio decile 6 is 1.197
Number of companies in year 2012 for portfolio decile 6 is 7
The average BE/ME in year 2012 for portfolio decile 7 is 1.351
Number of companies in year 2012 for portfolio decile 7 is 8
The average BE/ME in year 2012 for portfolio decile 8 is 1.548
Number of companies in year 2012 for portfolio decile 8 is 8
The avera

The average BE/ME in year 2019 for portfolio decile 1 is -1.412
Number of companies in year 2019 for portfolio decile 1 is 16
The average BE/ME in year 2019 for portfolio decile 2 is -0.582
Number of companies in year 2019 for portfolio decile 2 is 16
The average BE/ME in year 2019 for portfolio decile 3 is -0.291
Number of companies in year 2019 for portfolio decile 3 is 16
The average BE/ME in year 2019 for portfolio decile 4 is -0.101
Number of companies in year 2019 for portfolio decile 4 is 16
The average BE/ME in year 2019 for portfolio decile 5 is 0.072
Number of companies in year 2019 for portfolio decile 5 is 16
The average BE/ME in year 2019 for portfolio decile 6 is 0.239
Number of companies in year 2019 for portfolio decile 6 is 16
The average BE/ME in year 2019 for portfolio decile 7 is 0.419
Number of companies in year 2019 for portfolio decile 7 is 16
The average BE/ME in year 2019 for portfolio decile 8 is 0.669
Number of companies in year 2019 for portfolio decile 8 is

In [82]:
# Examine the average trading value for each decile portfolio in a given year
for year in yearSeries:
    beginPeriod = datetime(year.year, beginDate.month, beginDate.day)
    endPeriod = datetime(year.year+1, beginDate.month-1, beginDate.day)
    for decile in range(numPortfolio):
        myport = portfolios[year][decile]
        portfolioTradingVal = get_portfolio_yearly_avg_trading_val(myport, beginPeriod, endPeriod)
        stockDailyTradingVal = (portfolioTradingVal / 21) / len(myport)
        print(f"From year {year.year} to {year.year+1}, the average daily trading value for a stock in portfolio decile {decile+1} was {round(stockDailyTradingVal, 0):,}")

From year 2012 to 2013, the average daily trading value for a stock in portfolio decile 1 was 1,574,568,437.0
From year 2012 to 2013, the average daily trading value for a stock in portfolio decile 2 was 1,195,550,239.0
From year 2012 to 2013, the average daily trading value for a stock in portfolio decile 3 was 998,506,842.0
From year 2012 to 2013, the average daily trading value for a stock in portfolio decile 4 was 1,013,555,190.0
From year 2012 to 2013, the average daily trading value for a stock in portfolio decile 5 was 681,757,363.0
From year 2012 to 2013, the average daily trading value for a stock in portfolio decile 6 was 155,066,879.0
From year 2012 to 2013, the average daily trading value for a stock in portfolio decile 7 was 461,418,936.0
From year 2012 to 2013, the average daily trading value for a stock in portfolio decile 8 was 508,470,756.0
From year 2012 to 2013, the average daily trading value for a stock in portfolio decile 9 was 229,861,607.0
From year 2012 to 2013

From year 2020 to 2021, the average daily trading value for a stock in portfolio decile 2 was 2,452,309,752.0
From year 2020 to 2021, the average daily trading value for a stock in portfolio decile 3 was 1,844,327,896.0
From year 2020 to 2021, the average daily trading value for a stock in portfolio decile 4 was 1,760,855,194.0
From year 2020 to 2021, the average daily trading value for a stock in portfolio decile 5 was 1,178,563,647.0
From year 2020 to 2021, the average daily trading value for a stock in portfolio decile 6 was 5,627,490,292.0
From year 2020 to 2021, the average daily trading value for a stock in portfolio decile 7 was 1,246,561,330.0
From year 2020 to 2021, the average daily trading value for a stock in portfolio decile 8 was 1,486,482,952.0
From year 2020 to 2021, the average daily trading value for a stock in portfolio decile 9 was 961,986,908.0
From year 2020 to 2021, the average daily trading value for a stock in portfolio decile 10 was 1,591,912,887.0
From year 2

In [83]:
# Get monthly returns for each portfolio for each year
date = beginDate

portfolioReturns = {}
for year in yearSeries: # go inside year, having access to all portfolios
    nextYear = date.replace(year = date.year + 1)
    returnsInYear = []

    for i in range(len(portfolios[year])): # go inside to get monthly returns of individual portfolio
        temp = get_portfolio_monthly_return(portfolios[year][i], date, nextYear)
        returnsInYear.append(temp)
    portfolioReturns[year] = returnsInYear
    
    # Move to next year
    date = nextYear

In [84]:
# Calculate average returns of all months during testing period for decile portfolios
avgMonthlyPortfolioReturns = []

for i in range(numPortfolio):
    allYearsAvg = []
    for year in portfolioReturns:
        oneYearAvg = sum(portfolioReturns[year][i]) / len(portfolioReturns[year][i])
        allYearsAvg.append(oneYearAvg)
        print(f"In year {year.year}, portfolio decile {i+1} has an average monthly return of {round(oneYearAvg, 3)} percent")
    
    avgMonthlyPortfolioReturns.append(sum(allYearsAvg) / len(allYearsAvg))

In year 2012, portfolio decile 1 has an average monthly return of -0.353 percent
In year 2013, portfolio decile 1 has an average monthly return of 0.678 percent
In year 2014, portfolio decile 1 has an average monthly return of -0.333 percent
In year 2015, portfolio decile 1 has an average monthly return of 0.113 percent
In year 2016, portfolio decile 1 has an average monthly return of -0.775 percent
In year 2017, portfolio decile 1 has an average monthly return of -1.275 percent
In year 2018, portfolio decile 1 has an average monthly return of 0.028 percent
In year 2019, portfolio decile 1 has an average monthly return of -1.338 percent
In year 2020, portfolio decile 1 has an average monthly return of 2.025 percent
In year 2021, portfolio decile 1 has an average monthly return of 0.013 percent
In year 2012, portfolio decile 2 has an average monthly return of 2.935 percent
In year 2013, portfolio decile 2 has an average monthly return of 1.513 percent
In year 2014, portfolio decile 2 ha

In [85]:
avgMonthlyPortfolioBeta = get_portfolio_beta(portfolioReturns, dfMarketReturn)
avgMonthlyPortfolioStd = get_portfolio_std(portfolioReturns)

In [87]:
names = []
for portfolio in range(numPortfolio):
    names.append(f"Decile {str(portfolio)}")

dfReturns = pd.DataFrame({'Value Factor': names, 'Avg Monthly Return': avgMonthlyPortfolioReturns, 'Beta': avgMonthlyPortfolioBeta, 'Stdev': avgMonthlyPortfolioStd})
print(dfReturns)

  Value Factor  Avg Monthly Return   Beta  Stdev
0     Decile 0           -0.121568  0.779  5.611
1     Decile 1            0.825942  0.775  5.527
2     Decile 2            1.410461  0.757  6.372
3     Decile 3            1.092177  0.820  6.266
4     Decile 4            1.797587  0.824  6.455
5     Decile 5            1.545527  0.829  7.050
6     Decile 6            1.614495  0.982  8.145
7     Decile 7            1.796280  0.979  7.704
8     Decile 8            2.290880  1.000  8.997
9     Decile 9            4.077941  1.051  9.834


# Print out the returns


In [88]:
dfMonthlyReturns = pd.DataFrame()
dfCumReturns = pd.DataFrame()

In [89]:
for portfolio in range(numPortfolio):
    data = []
    for year in portfolioReturns: 
        for month in portfolioReturns[year][portfolio]:
            data.append(month)
    dfMonthlyReturns["Decile " + str(portfolio + 1)] = pd.Series(data)

dfMonthlyReturns['months'] = pd.date_range(start=beginDate.replace(month = beginDate.month + 1), end=endDate, freq = 'M')
dfMonthlyReturns = dfMonthlyReturns.set_index('months')
print(dfMonthlyReturns)

            Decile 1  Decile 2   Decile 3   Decile 4   Decile 5   Decile 6  \
months                                                                       
2012-07-31 -4.744147  0.620004  -1.389180  -0.129448  -1.641701   6.217880   
2012-08-31 -6.343790  1.813975  -6.316037  -9.289801  -5.671100  -6.805565   
2012-09-30 -7.482162 -2.167782  -7.643093 -11.247408  -3.942383  -4.449370   
2012-10-31 -2.972114  7.359854  -0.738955   1.861167  -3.888530  -3.913228   
2012-11-30 -7.644711  1.668220  -3.810247  -6.275404   7.127060   2.812761   
...              ...       ...        ...        ...        ...        ...   
2022-02-28  3.834496  4.045484  10.489537   9.864695   9.288346  18.690370   
2022-03-31  2.524317 -0.145179   6.676840   7.858131   3.750075   4.072112   
2022-04-30 -4.791756 -9.874987 -11.774297  -8.246802 -10.772993 -10.805372   
2022-05-31 -5.080889 -6.402320  -7.403645  -5.673961  -4.149823  -9.875096   
2022-06-30 -3.853745 -8.603079  -9.533121  -8.559687  -9.877328 

In [90]:
for portfolio in range(numPortfolio):
    data = []
    for year in portfolioReturns: 
        for month in portfolioReturns[year][portfolio]:
            data.append(month)
    dfMonthlyReturns["Decile " + str(portfolio + 1)] = pd.Series(data)
    
    cumReturn = []
    totalReturn = 100
    for i in range(len(data)):
        totalReturn = totalReturn * (1 + data[i]/100)
        cumReturn.append(totalReturn)
        
    dfCumReturns["Decile " + str(portfolio + 1)] = pd.Series(cumReturn)
print(dfCumReturns)

      Decile 1    Decile 2    Decile 3    Decile 4    Decile 5    Decile 6  \
0    95.255853  100.620004   98.610820   99.870552   98.358299  106.217880   
1    89.213021  102.445226   92.382524   90.592776   92.780301   98.989153   
2    82.537958  100.224436   85.321641   80.403437   89.122546   94.584759   
3    80.084836  107.600809   84.691152   81.899880   85.656988   90.883442   
4    73.962582  109.395826   81.464210   76.760332   91.761813   93.439776   
..         ...         ...         ...         ...         ...         ...   
115  80.477933  291.050532  538.336210  344.155471  833.953752  630.204217   
116  82.509451  290.627987  574.280059  371.199660  865.227647  655.866837   
117  78.555800  261.928512  506.662619  340.587557  772.016735  584.997987   
118  74.564467  245.159011  469.151119  321.262752  739.979405  527.228874   
119  71.690943  224.067788  424.426377  293.763665  666.889209  474.695807   

       Decile 7    Decile 8     Decile 9     Decile 10  
0     